In [1]:
#imports
import eqcorrscan
from eqcorrscan import Tribe
import pandas as pd
import numpy as np
import yaml
from glob import glob
from obspy import UTCDateTime

Matched-filter CPU is not compiled! Should be here: /home/jupyter_share/miniconda3/envs/seismo/lib/python3.8/site-packages/fast_matched_filter/lib/matched_filter_CPU.so


In [2]:
with open('/home/smocz/expand_redpy/scripts/config.yaml') as file:
    config = yaml.load(file, Loader=yaml.FullLoader)
    
readdir = config['readdir']
homedir = config['homedir']

# find X
29,404 number of templates from X number of waveforms. 

In [3]:
#read in REDPy catalogs and metadata
Baker = pd.read_csv(readdir+'Baker_catalog.csv')
Hood = pd.read_csv(readdir+'Hood_catalog.csv')

St_Helens = pd.read_csv(readdir+'MountStHelens_catalog.csv')

# Combining borehole and local catalogs with St_Helens

Helens_Borehole = pd.read_csv(readdir+'MSHborehole_catalog.csv')
Helens_Borehole['Clustered'] += 2000 
# Cluster 0 in Helens_Borehole is now Cluster 2000 in St_Helens
Helens_Local = pd.read_csv(readdir+'MSHlocal_catalog.csv')
Helens_Local['Clustered'] += 3000
# Cluster 0 in Helens_Local is now Cluster 3000 in St_Helens

# Use St_Helens to access all three St Helens catalogs
St_Helens = pd.concat([St_Helens,Helens_Borehole,Helens_Local])
clid = np.unique(St_Helens['Clustered'].values.tolist()) #find the largest cluster ID for a volcano to set range
print(clid[-1])

Newberry = pd.read_csv(readdir+'Newberry_catalog.csv')
Rainier = pd.read_csv(readdir+'Rainier_catalog.csv')

volc_md = pd.read_csv(readdir+'Volcano_Metadata.csv')
volc_md['netsta'] = volc_md['Network'].astype(str)+'.'+volc_md['Station'].astype(str)
# read metadata file to create dataframe of labels

#list of catalogs
volc_dict = {'Baker': Baker,'Hood': Hood,'Newberry': Newberry,'Rainier': Rainier,'St_Helens': St_Helens}

3523


In [ ]:
#number of REDPy events for each cluster on each station for the amount of time the station was active

cl_event_dict = {} #dictionary, keys = clusterIDs, value = number of events on that cluster

events_per_station = []
for index, row in volc_md.iterrows(): #through each station (row of volc_md)
    volc_name = row['Volcano_Name']
    net = row['Network']
    sta = row['Station']
    #call tgz file
    try:
        T = Tribe().read(*glob(f'{homedir}templates/Volcano_{volc_name}_Network_{net}_Station_{sta}_Channel_*.tgz'))
    except:
        print(f'{net}.{sta} tgz does not exist')
        print(*glob(f'{homedir}templates/Volcano_{volc_name}_Network_{net}_Station_{sta}_Channel_*.tgz'))
        continue
    t_amt = len(T) #number of templates, inherently accounts for time station was active
    #duration of station:
    starttime = UTCDateTime(row['Starttime']) #starttime for the station
    endtime = UTCDateTime(row['Endtime']) #endtime for the station

    events_per_cluster = []
    for t in T:
        #get clusterID
        if volc_name=='Baker' or volc_name=='Hood' or volc_name=='Newberry' or volc_name=='Rainier':
            cl = t.name[-3:] #account for zfill, to parameterize, use what make_templates uses to get zfill amount (cid[-1] from volcmd)
        if volc_name=='St_Helens':
            cl = t.name[-4:] #account for zfill
        
#         print('---',cl)
        
        catalog = volc_dict[volc_name]
        catalog_events = catalog[catalog['Clustered']==int(cl)]['datetime'].values.tolist()#list of event times for this clusterID #from REDPy catalog
        cl_events = [UTCDateTime(i) for i in catalog_events] #to datetime
#         print(f'original | len: {len(cl_events)} |')
        for event in cl_events:
            if event < starttime:
                cl_events.remove(event)#remove it
#                 print('removed')
                continue #skip because it is already removed
            if event > endtime:
                cl_events.remove(event)#remove it
#                 print('removed')
#         print(f'updated | len: {len(cl_events)} |')
        amt_events = len(cl_events) #length of cl_events after removals
        events_per_cluster.append(amt_events)
        print(amt_events,end=' ')
    print(' ')
    events_per_station.append(sum(events_per_cluster))
    print(f'events per station {net}.{sta}',events_per_station)
#     break
    
total_events = sum(events_per_station)
print(total_events)

In [9]:
print(total_events)

369655


#------#

total events = 369655 = X

#------#

 # find Y
 To do Y cross-correlations (minimum estimate)

In [ ]:
# of cross correlations per day:
cc_per_chunk = round(200/ 78) #if overlap is minimal, how many cross correlation per chunk (of 200s)
print(cc_per_chunk)
chunks_per_day = 86400 / 200 #how many chunks per day

min_cc_per_day = chunks_per_day*cc_per_chunk
print(min_cc_per_day)


list_of_cc_per_station = []
for index, row in volc_md.iterrows(): #through each station (row of volc_md)
    volc_name = row['Volcano_Name']
    net = row['Network']
    sta = row['Station']
    print('----',volc_name,net,sta)
    #call tgz file
    try:
        T = Tribe().read(*glob(f'{homedir}templates/Volcano_{volc_name}_Network_{net}_Station_{sta}_Channel_*.tgz'))
    except:
        print(f'{net}.{sta} tgz does not exist')
        print(*glob(f'{homedir}templates/Volcano_{volc_name}_Network_{net}_Station_{sta}_Channel_*.tgz'))
        continue
    t_amt = len(T)#number of templates for that station, from .tgz file
    
    starttime = UTCDateTime(row['Starttime']) #starttime for the station
    if starttime < UTCDateTime('2002-01-01T00:00:00'): #only account for time I'm running over, not the entire duration
        starttime = UTCDateTime('2002-01-01T00:00:00')
    endtime = UTCDateTime(row['Endtime']) #endtime for the station
    if endtime > UTCDateTime('2022-01-01T00:00:00'): #only account for time I'm running over, not the entire duration
        endtime = UTCDateTime('2022-01-01T00:00:00')
        
    sta_len = (endtime-starttime)/86400 #station duration in s / (s/day) = station duration in days
    cc_per_sta = t_amt*sta_len*min_cc_per_day #templates to run * number of days to run over * cc per day
    list_of_cc_per_station.append(cc_per_sta)
    
    
total_cc = sum(list_of_cc_per_station)
print(total_cc)

In [5]:
print(total_cc)

184156214487.41998


#------#

minimum total cc calculations = 184156214487 = Y

#------#